In [ ]:
# Information necessary to grab API (Application Programming Interface)
username ='OOIAPI-NAL2K3VMHB06OA'
token = 'TEMP-TOKEN-68QRXA825THDH4'
API_BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

In [ ]:
# First, let's import the Python libraries we will need
import requests
import datetime
import matplotlib.pyplot as plt

In [ ]:
import requests
import time

In [ ]:
# Input the Instrument Information for the instrument of interest (can be found on the OOI data website)
subsite = 'GA01SUMO'  #the site of the insturment?
node = 'SBD12'    
sensor = '05-WAVSSA000'
method = 'recovered_host'
stream = 'wavss_a_dcl_statistics_recovered' 
beginDT = '2015-03-15T21:25:00.000Z' 
endDT = '2018-01-14T10:24:00.000Z'





In [ ]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,   
}
r = requests.get(data_request_url, params=params, auth=(username, token))
data = r.json()

In [ ]:
data['allURLs'][0]

In [ ]:
!pip install xarray
!pip install netCDF4
!pip install bokeh
!pip install dask

In [ ]:
%%time
check_complete = data['allURLs'][1] + '/status.txt'
for i in range(1800): 
    r = requests.get(check_complete)
    if r.status_code == requests.codes.ok:
        print('request completed')
        break
    else:
        time.sleep(1)

In [ ]:
import re
import xarray as xr
import pandas as pd
import os

In [ ]:
#Parse the thredds server to get a list of all NetCDF files. Each deployment is seperated into a seperate netcdf file.

url = data['allURLs'][0]
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]
datasets

In [ ]:
# load all the files into a single xarray data set

ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.chunk({'time': 100})
ds = ds.sortby('time') # data from different deployments can overlap so we want to sort all data by time stamp.
ds


In [ ]:
ds2 = ds.sel(time=slice('2016-03-01', '2016-11-01'))
ds2.time

In [ ]:
import pandas as pd

# find the time between values i.e. recording frequency
df = ds.to_dataframe()
res = (pd.Series(df.index[1:]) - pd.Series(df.index[:-1])).value_counts()
res

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# extract just the data into an array that can then be manipulated

wv_d = ds['mean_direction'].values
Hs = ds['significant_wave_height'].values
Hmax = ds['max_wave_height'].values
t = ds['time'].values
wv_p = ds['mean_wave_period'].values    
wv_spd = ds['mean_spread'].values


In [ ]:
# extract just the data into an array that can then be manipulated

wv_d2 = ds2['mean_direction'].values
Hs2 = ds2['significant_wave_height'].values
Hmax2 = ds2['max_wave_height'].values
t2 = ds2['time'].values

In [ ]:
#plot histotgram of signficant wave height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

#ds['significant_wave_height'].plot(linestyle = 'None', 
#                                   marker='.', 
#                                   markersize=1,
#                                   ax=ax)

ds['significant_wave_height'].plot.hist(bins=50)
                                 

In [ ]:
#plot signficant wavve height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['max_wave_height'].plot()
ds['significant_wave_height'].plot()
plt.xlabel('Date')
plt.ylabel('Wave Height[m]')




In [ ]:
#plot signficant wavve height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['max_wave_height'].plot()
plt.xlabel('Date')
plt.ylabel('Wave Height[m]')


In [ ]:
#plot signficant wavve height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['significant_wave_height'].plot()
plt.xlabel('Date')
plt.ylabel('Wave Height[m]')


In [ ]:
#plot signficant wavve height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['mean_direction'].plot()
plt.xlabel('Date')
plt.ylabel('Wave Direction')

In [ ]:
#plot signficant wavve height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['mean_spread'].plot()
plt.xlabel('Date')
plt.ylabel('Wave Spread')

In [ ]:
#plot peak wave period

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['peak_wave_period'].plot()
plt.xlabel('Date')
plt.ylabel('Peak_wave_period')

In [ ]:
#plot signficant wavve height

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['number_zero_crossings'].plot()
plt.xlabel('Date')
plt.ylabel('number of zero crossings')



In [ ]:
# calculate abnormality index

AI = Hmax/Hs
AI[0]


In [ ]:
plt.scatter(t,AI)
plt.title('Global Argentine Basin Signficant Wave Height Abnormality Index')
plt.ylabel('Abnormality Index')
plt.xlabel('Date')

In [ ]:
#plot histotgram of signficant wave height may-nov 2015

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)



plt.hist(Hs[0:5870],bins = 50)

plt.title('Global Argentine Basin Signficant Wave Height Deployment May-Nov 2015 and 2016')
plt.ylabel('Counts')
plt.xlabel('Wave Height [m]')
# plt.show()

#plot histotgram of signficant wave height may-nov 2016


plt.hist(Hs2,bins = 50,color="red",alpha = 0.5)



In [ ]:
#plot histotgram of maximum wave height may-nov 2015

fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)



plt.hist(Hmax[0:5870],bins = 50)

plt.title('Global Argentine Basin Maximum Wave Height Deployment May-Nov 2015 and 2016')
plt.ylabel('Counts')
plt.xlabel('Wave Height [m]')
# plt.show()

#plot histotgram of signficant wave height may-nov 2016


plt.hist(Hmax2,bins = 50,color="red",alpha = 0.5)



In [ ]:
ds3 = ds.sel(time=slice('2017-03-21', '2017-4-01'))
ds3.time

In [ ]:
# extract just the data into an array that can then be manipulated

wv_d3 = ds3['mean_direction'].values
Hs3 = ds3['significant_wave_height'].values
Hmax3 = ds3['max_wave_height'].values
t3 = ds3['time'].values
AI3 = Hmax3/Hs3


In [ ]:
plt.scatter(t3,AI3)
plt.title('Global Argentine Basin Signficant Wave Height Abnormality Index')
plt.ylabel('Abnormality Index')
plt.xlabel('Date')

In [ ]:
plt.scatter(t3,Hmax3)
plt.scatter(t3,AI3)
plt.title('Global Argentine Basin Signficant Wave Height Abnormality Index')
plt.ylabel('Abnormality Index')
plt.xlabel('Date')
%plt.scatter(t3,Hs3)

In [ ]:
plt.plot(t,wv_d)
plt.title('Global Argentine Basin Signficant Wave Height Abnormality Index')
plt.ylabel('Abnormality Index')
plt.xlabel('Date')

In [ ]:
plt.plot(t,wv_spd)
plt.title('Global Argentine Basin Signficant Wave Height Abnormality Index')
plt.ylabel('Abnormality Index')
plt.xlabel('Date')



In [ ]:

plt.scatter(wv_d,Hs)
plt.title('Wave Height As a Function of Direction')
plt.ylabel('Signficant Wave Height')
plt.xlabel('Wave Direction')
